In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer,GPT2LMHeadModel, get_scheduler 
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/arjun/NewPytorchEnv/bin/python3.10: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizer

/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('0'), PosixPath('1')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('@/tmp/.ICE-unix/1600,unix/PC'), PosixPath('local/PC')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path wer

Parameters

In [2]:
num_epochs = 1 
lr = 5e-5
batch_size = 1
save_loc = '/home/arjun/Documents/ModelSaves/GPT2Alpaca'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


Dataset preparation and tokenising

In [3]:
dataset = load_dataset("tatsu-lab/alpaca")

Using custom data configuration tatsu-lab--alpaca-2b32f0433506ef5f
Found cached dataset parquet (/home/arjun/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset = dataset['train']
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [5]:
# Making dataset smaller for fast training
dataset = dataset.select(range(100))

In [6]:
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

In [7]:
tokenizer = AutoTokenizer.from_pretrained(save_loc)
model =   GPT2LMHeadModel.from_pretrained(save_loc)

In [8]:
new_dataset = {'input_ids':[], 'attention_mask':[]}

for example in dataset:
    input_text = example['text'].replace('###','')   
    encoded_data = tokenizer('<|startoftext|>' + input_text + '<|endoftext|>',truncation=True, max_length=768, padding="max_length")
    new_dataset['input_ids'].append(encoded_data['input_ids'])
    new_dataset['attention_mask'].append(encoded_data['attention_mask'])

new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")

DataLoader

In [9]:
dataloader = DataLoader(new_dataset,shuffle=True, batch_size=batch_size)

Optimiser and scheduler

In [10]:
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [11]:
model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50259, bias=False)
)

Gradient accumulation for training loop

In [12]:
'''
This is gradient accumulation.
The idea is to put a low batch size(say, 1) and a high gradient_accumulation step(say, 8) instead
of taking a batch size of 8

This will reduce memory footprint by a lot, and retain the performance(with a small loss)
But the training time increases a lot

Probably useful for very large LLMs.

'''

from torch import nn

# Define the gradient accumulation step size
gradient_accumulation_steps = 8  # Choose an appropriate value based on your GPU memory

progress_bar = tqdm(range(num_training_steps-1), desc='Training', unit='steps')
model.train()
ep = 0
total_train_loss = 0
optimizer.zero_grad()  # Move optimizer.zero_grad() outside the epoch loop

for epoch in range(num_epochs):
    for i, batch in enumerate(dataloader):
        batch_data = batch['input_ids'].to(device)
        attention = batch['attention_mask'].to(device)
        
        outputs = model(batch_data,
                        labels=batch_data,
                        attention_mask=attention,
                        token_type_ids=None
                        )

        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Perform backward pass and accumulate gradients
        loss = loss / gradient_accumulation_steps
        loss.backward()

        progress_bar.update(1)
        if (i + 1) % gradient_accumulation_steps == 0:
            # Update parameters after accumulating gradients for the specified number of steps
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()


    avg_train_loss = total_train_loss / len(dataloader)
    ep += 1
    print('Epoch:', ep, 'Average training loss =', avg_train_loss)
    total_train_loss = 0  # Reset the total training loss for the next epoch


Training:   0%|          | 0/99 [00:00<?, ?steps/s]

Epoch: 1 Average training loss = 0.1841659372858703


In [13]:
question = 'Write a poem on cow'

prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
 Instruction:{question}
 Response: """
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=100,
                                max_length=100,
                                max_new_tokens=200,
                                top_p=.95, 
                                num_return_sequences= 5,
                                temperature = .9,
                                )

for i, sample_output in enumerate(sample_outputs):
    ans = tokenizer.decode(sample_output, skip_special_tokens=True).split('Response: ')
    print("\n\n-------------------------------------------------------------------------------------------------------------------------------------------")
    try:        print(f'<-{i+1}-> {ans[1]}')
    except:
        print(f'<-{i+1}-> ___No response___')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




-------------------------------------------------------------------------------------------------------------------------------------------
<-1->   

Answer:
She lived on the shore, where the rivers gave her away her beauty.
 She was beloved by all, and she was beloved by every creature.
 As soon as she was released, her beauty grew and she became more beautiful.
  
 


-------------------------------------------------------------------------------------------------------------------------------------------
<-2-> 
       
 Instruction:        
 


-------------------------------------------------------------------------------------------------------------------------------------------
<-3-> 
I love to go

I don't like to go

I don't like to be

I haven't even thought of it yet

I haven't even thought about it

It's too late and I haven't even thought of it yet




-------------------------------------------------------------------------------------------------------------------------